In [211]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs, Statistics, Random, LinearAlgebra
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using TensorTrains: summary_compact
using SparseArrays;

In [212]:
T = 20
N = 30
seed = 4

c = 4
gg = erdos_renyi(N, c/N; seed)
g = IndexedGraph(gg)

λ_unif = 0.8
ρ_unif = 0.6
λ = zeros(N,N)
for i in CartesianIndices(λ)
    if !iszero(g.A[i])
        λ[i] = rand()
        # λ[i] = λ_unif
    end
end
λ = sparse(λ)
ρ = rand(N)
# ρ = fill(ρ_unif,N)
γ = 0.8;

# T = 7
# N = 2
# seed = 6

# A = [0 1; 1 0]
# g = IndexedGraph(A)

# λ_unif = 0.7
# ρ_unif = 0.6
# λ = sparse(λ_unif .* A)
# # λ = sparse([0 1e-12; λ_unif 0])
# ρ = fill(ρ_unif, N)
# γ = 0.5;
;

In [213]:
sis = SIS_heterogeneous(λ, ρ, T; γ);
bp_obs = mpbp(sis);

In [214]:
g.A

30×30 SparseMatrixCSC{Int64, Int64} with 134 stored entries:
⠀⠀⠀⢢⠤⡀⢡⠂⠁⠐⠴⡀⠀⢠⠠
⠠⣀⠀⠀⠀⡀⢈⡀⠀⢀⠀⡀⠛⠄⠐
⠀⠣⠀⠠⢀⠐⠀⠠⠈⠀⠂⠠⢁⠁⠀
⠡⠒⠂⠰⠀⡀⠊⠀⠥⠀⠀⠁⠈⠤⠅
⢁⠀⠀⢀⠂⠀⠁⠃⡀⡨⡁⠀⠒⠁⠁
⠐⠣⠀⠠⠈⡀⠄⠀⠁⠈⠀⡠⣉⠠⠐
⠀⣀⠛⠄⠅⠐⠂⡄⠜⠀⠃⡘⢀⠐⠒
⠀⠂⠐⠀⠀⠀⠁⠁⠁⠀⠐⠀⠘⠀⠀

In [215]:
obs_times = collect(0:T)
nobs = floor(Int, N * length(obs_times) * 1.0)
obs_fraction = nobs / N
rng = MersenneTwister(seed)
X, observed = draw_node_observations!(bp_obs, nobs, times = obs_times .+ 1, softinf=Inf; rng);

In [216]:
X

30×21 Matrix{Int64}:
 2  1  1  2  1  2  1  2  1  1  2  1  2  1  1  1  2  2  2  2  2
 2  1  1  2  1  2  1  1  1  2  1  2  1  1  2  1  2  1  2  1  1
 2  1  2  2  2  2  2  2  2  2  2  1  2  2  2  1  2  2  2  2  2
 2  2  2  2  1  2  2  2  2  2  2  1  2  1  2  1  2  1  1  2  1
 2  2  2  2  1  2  2  2  2  2  2  2  2  2  1  2  1  2  2  2  1
 2  2  1  2  2  2  2  1  1  2  2  2  2  2  2  2  1  2  1  2  2
 2  2  2  2  2  2  1  1  1  2  1  1  2  1  1  2  1  2  1  2  1
 2  1  2  1  2  1  2  1  2  1  2  1  2  2  1  2  1  2  2  1  2
 2  2  2  1  2  2  1  1  2  2  1  2  2  2  2  1  2  1  2  2  2
 2  1  2  1  1  2  1  2  2  2  1  2  1  1  1  1  1  2  2  1  2
 ⋮              ⋮              ⋮              ⋮              ⋮
 2  2  2  1  2  2  1  1  2  1  2  2  1  2  1  2  1  2  2  1  1
 2  2  2  1  2  1  2  1  2  1  2  1  2  1  2  1  2  2  1  1  2
 1  2  1  2  1  2  1  2  1  2  1  2  1  2  1  2  1  2  1  2  1
 2  2  2  2  2  1  2  2  2  2  2  1  2  2  2  2  2  2  2  2  2
 2  2  1  2  2  2  2  2  2  1  2  

In [217]:
λinit = 0.5
ρinit = 0.5

A_complete = ones(N,N) - I
g_complete = IndexedGraph(A_complete)
λ_complete = sparse(λinit.*A_complete)
ρ_complete = fill(ρinit, N)

sis_inf = SIS_heterogeneous(g_complete, λ_complete, ρ_complete, T; γ, ϕ=deepcopy(bp_obs.ϕ))
bp_inf = mpbp(sis_inf);

In [218]:
svd_trunc = TruncBond(10)

iters, cb = inference_parameters!(bp_inf, method=2, maxiter=10, λstep=0.01, ρstep=0.01);

In [219]:
@show cb.data

11-element Vector{PARAMS{Float64}}:
 PARAMS{Float64}([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5  …  0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], [